In [1]:
import matplotlib.pyplot as plt
import numpy as np

ImportError: cannot import name 'plt' from 'matplotlib.pyplot' (/Users/mitchellblack/Documents/WashU SP21/ESE 417 Intro to Machine Learning and Pattern Classification/Final Project/venv/lib/python3.7/site-packages/matplotlib/pyplot.py)

In [ ]:
# Read the data